In [15]:
import shapely
import requests
import geopandas as gpd
import numpy as np
import pandas as pd

path = 'https://restoration-monitoring-external.s3.amazonaws.com/Dummy_Polygon.geojson'
user = None
password = None
fname = path.split("/")[-1]

# If you need a password to access the file, then login with this
if user is not None:
    r = requests.get(path, auth=(user, password))
# Otherwise, just download the link
else:
    r = requests.get(path)
    
# Save the data to a file    
with open(fname, 'wb') as f:
    f.write(r.content)

# Open the file an set the CRS to 4326, then reproject it to equal-area 6933
data = gpd.read_file(fname)
countryfile = gpd.read_file('countries.geojson')
data = gpd.sjoin(data, countryfile, predicate = 'within')

df = pd.DataFrame()
data.crs = {"init":"epsg:4326"}
data = data.to_crs(epsg = 6933)
for index, polygon in data.iterrows():
    print(f'Polygon {index} is in {polygon.ADMIN}  with a size of '
          f'{np.around(polygon.geometry.area / 1e4, 1)} ha')
    df = df.append({'name': fname,
                    'polygon': index,
                    'area': np.around(polygon.geometry.area / 1e4, 2),
                    'country': polygon.ADMIN,
                    # Here you could add any columns from the polygon attributes
                   }, ignore_index = True)
    
df.head(5)

Polygon 0 is in Australia  with a size of 66.6 ha
Polygon 1 is in Australia  with a size of 5.2 ha
Polygon 2 is in Australia  with a size of 70.4 ha


/Users/jbrandt.terminal/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,area,country,name,polygon
0,66.56,Australia,Dummy_Polygon.geojson,0.0
1,5.16,Australia,Dummy_Polygon.geojson,1.0
2,70.39,Australia,Dummy_Polygon.geojson,2.0
